In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy.stats as st
import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_Frame_repr", False)
pd.set_option("display.float_format", lambda x: "%.4f" % x)

In [5]:
df = pd.read_csv("amazon_review.csv")

In [9]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0000,best deal around,1373673600,2013-07-13,513,0,0


### Görev 1 : Average rating güncel yorumlara göre hesaplayınız.

In [12]:
df["overall"].mean()

4.587589013224822

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


#### Son 30 günde

In [17]:
df.loc[df["day_diff"] < 30, "overall"].mean()

4.742424242424242

In [23]:
df["day_diff"].min(), df["day_diff"].max()

(1, 1064)

In [25]:
df["day_diff"].mean()

437.3670396744659

In [29]:
df.loc[df["day_diff"] < 250, "overall"].mean() *30/100 + \
df.loc[(df["day_diff"] >= 250) & (df["day_diff"] < 500), "overall"].mean() * 28/100 + \
df.loc[(df["day_diff"] >= 500) & (df["day_diff"] < 750), "overall"].mean() * 22/100 + \
df.loc[df["day_diff"] >= 750, "overall"].mean() * 20/100 

4.5709849626887245

In [31]:
df["overall"].mean()

4.587589013224822

#### Değerler birbirine yakın çıktı:

## Görev 2: Ürün için ürün detay sayfasında görüntülenecek 20 rewievi belirleyiniz

#### Adım 1: helpful_no değişkenini bulunuz

In [45]:
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

In [53]:
df["helpful_no"].sum(), df["helpful_yes"].sum(), df["total_vote"].sum()

(1034, 6444, 7478)

#### Adım 2:Bazı skorları bulup hesaplayınız.(wilson_lower, score_Average, score_pos_neg)

In [56]:
def score_pos_neg_diff(up, down):
    return up - down

In [60]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [62]:
def wilson_lower_bound(up, down, confidence=0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1-(1-confidence) /2)
    phat = 1.0 * up / n
    return (phat + z**2 / (2 * n) - z * math.sqrt((phat * (1 - phat) + z**2 / (4 * n)) / n)) / (1 + z**2 / n)


In [76]:
df["score_pos_neg_diff"] = df.apply(lambda x: score_pos_neg_diff(x["helpful_yes"], x["helpful_no"]), axis=1)

In [78]:
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)

In [80]:
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)

In [88]:
#İlk 20 yorum
df.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.0000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,68,1884,0.9663,0.9575
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.0000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,77,1351,0.9488,0.9365
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.0000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,126,1442,0.9256,0.9121
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.0000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,73,349,0.8525,0.8186
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.0000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,4,41,0.9184,0.8081
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.0000,I own it,1393545600,2014-02-28,283,60,68,8,52,0.8824,0.7847
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.0000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,27,85,0.8058,0.7321
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.0000,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25,3,19,0.8800,0.7004
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.0000,Awesome Card!,1339200000,2012-09-06,823,51,65,14,37,0.7846,0.6703
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.0000,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,27,55,0.7523,0.6636
